In [2]:
import logging
import torchaudio
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from transformers import WhisperProcessor, WhisperFeatureExtractor, WhisperTokenizer, WhisperForConditionalGeneration, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
from jiwer import wer
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score
from transformers import TrainerCallback


import pydub
import numpy as np
import os
import csv
import json
from pydub.utils import mediainfo
from pydub import AudioSegment

C:\Users\elmog\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tiny_model_directory = "./models/whisper-tiny.en"
# processor = WhisperProcessor.from_pretrained(tiny_model_directory, language="English", task="transcribe", chunk_length=3)
# extractor = WhisperFeatureExtractor.from_pretrained(tiny_model_directory, language="English", task="transcribe", chunk_length=3)
processor = WhisperProcessor.from_pretrained(tiny_model_directory)
extractor = WhisperFeatureExtractor.from_pretrained(tiny_model_directory)
model = WhisperForConditionalGeneration.from_pretrained(tiny_model_directory)
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.to("cuda")

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 384, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(384, 384, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 384)
      (layers): ModuleList(
        (0-3): 4 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=384, out_features=384, bias=False)
            (v_proj): Linear(in_features=384, out_features=384, bias=True)
            (q_proj): Linear(in_features=384, out_features=384, bias=True)
            (out_proj): Linear(in_features=384, out_features=384, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (fin

In [5]:
logging.basicConfig(level=logging.INFO)

# Initialize an empty list to hold the new data dictionaries
new_data_dicts = []

with open('new_data.json', 'r') as json_file:
    new_data_dicts = json.load(json_file)
    
    # Loop through each entry in the list of dictionaries
    for data_dict in new_data_dicts:
        print("sampling & creating dataset row:")
        path = data_dict["path"]
        transcription = data_dict["transcription"]
        
        # Here, audio information is already available in data_dict["audio"]
        new_sampling_rate = data_dict["audio"]["sampling_rate"]
        audio_data = np.array(data_dict["audio"]["array"])
print(new_data_dicts[:1])


sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling & creating dataset row:
sampling &

In [6]:
logging.info("Starting to load and prepare data.")
df = pd.json_normalize(new_data_dicts)
logging.info("Data converted to Pandas DataFrame.")

train_df, temp_df = train_test_split(df, test_size=0.1, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


logging.info(f"Data split into train, validation, and test sets. Train: {len(train_df)}, Validation: {len(val_df)}, Test: {len(test_df)}")

# Create Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

dataset_dict = DatasetDict({"train": train_dataset, "validation": val_dataset, "test": test_dataset})

INFO:root:Starting to load and prepare data.
INFO:root:Data converted to Pandas DataFrame.
INFO:root:Data split into train, validation, and test sets. Train: 2290, Validation: 127, Test: 128


In [7]:
# # Fetch a single batch to get data shape
# print("Fetching a single batch to get data shape...")
# sample_batch = next(iter(train_dataset))
# print("Sample batch fetched.")

# print("Preparing the sample batch...")
# prepared_sample_batch = prepare_dataset(sample_batch)
# print("Sample batch prepared.")

# print("Fetching 'input_features' shape...")
# input_data_shape = prepared_sample_batch['input_features'].shape
# print(f"'input_features' shape: {input_data_shape}")

# # Convert 'labels' to tensor and then get its shape
# print("Converting 'labels' to tensor...")
# labels_tensor = torch.tensor(prepared_sample_batch['labels'])
# print(f"'labels' tensor: {labels_tensor}")

# print("Fetching 'labels' shape...")
# decoder_input_shape = labels_tensor.shape
# print(f"'labels' shape: {decoder_input_shape}")

# # Set the device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Device set to: {device}")

# # Move the model to the device
# print("Moving the model to the device...")
# model.to(device)
# print("Model moved to device.")

# # Find max batch size
# print("Starting the search for maximum batch size...")
# max_batch_size = find_max_batch_size(model, input_data_shape, device, decoder_input_shape)
# print(f"The maximum batch size that can fit in memory is {max_batch_size}")


In [8]:
# Assuming you have a function to get the shape of a single data point
# replace 'data_shape' with the shape of a single input data point

# Find the maximum batch size that can fit into the GPU


In [9]:
# from collections import Counter
# ######
# ######
# ######
# #####Plot top 20 most repeated words
# # Assuming dataset_dict["train"]["transcription"] contains all the transcription texts
# print(dataset_dict["train"]["transcription"][:5])
# all_transcriptions = dataset_dict["train"]["transcription"]

# # Create one big list of words from all transcriptions
# all_transcriptions_tuple = tuple(all_transcriptions)
# print(all_transcriptions_tuple[:5])

# # Count the occurrences of each word
# word_counter = Counter(all_transcriptions_tuple)

# # Get the top 30 most common words and their counts
# most_common_30 = word_counter.most_common(30)

# # Separate the words and their counts into two lists
# words, counts = zip(*most_common_30)

# # Text output
# print("Top 30 most common words:")
# for word, count in most_common_30:
#     print(f"{word}: {count}")

# # Plotting
# plt.figure(figsize=(12, 8))
# plt.barh(words, counts, color='skyblue')
# plt.xlabel('Count')
# plt.ylabel('Word')
# plt.title('Top 20 Most Common Words')
# plt.gca().invert_yaxis()  # Reverse the order for better readability
# plt.show()

In [10]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio_array = batch['audio.array']
    audio_sampling_rate = batch['audio.sampling_rate']

    # compute log-Mel input features from input audio array 
    #batch["input_features"] = processor.feature_extractor(audio_array, sampling_rate=audio_sampling_rate, chunk_length=5).input_features[0]
    # extractor.hop_length = 160 # Update dep/endent attribute
    # extractor.chunk_length = 3
    # extractor.n_samples = extractor.chunk_length * extractor.sampling_rate  # Update dependent attribute
    # extractor.nb_max_frames = extractor.n_samples // extractor.hop_length
    print("extractor.chunk_length: ",extractor.chunk_length)
    print("extractor.sampling_rate: ",extractor.sampling_rate)
    print("extractor.n_samples: ",extractor.n_samples)
    print("extractor.nb_max_frames: ",extractor.nb_max_frames)
    print("extractor.hop_length: ",extractor.hop_length)
    
    # batch["input_features"] = extractor(audio_array, sampling_rate=audio_sampling_rate, chunk_length=3).input_features[0]
    batch["input_features"] = extractor(audio_array, sampling_rate=audio_sampling_rate, chunk_length=3).input_features[0]

    # encode target text to label ids 
    batch["labels"] = processor.tokenizer(batch["transcription"]).input_ids
    return batch

In [11]:
print("Columns in train set before:", dataset_dict["train"].column_names)
dataset_dict = dataset_dict.map(prepare_dataset, remove_columns=dataset_dict.column_names["train"])
print("Columns in train set after:", dataset_dict["train"].column_names)

Columns in train set before: ['path', 'transcription', 'audio.path', 'audio.sampling_rate', 'audio.array', '__index_level_0__']


Map:   0%|                             | 3/2290 [00:00<02:01, 18.80 examples/s]

extractor.chunk_length:  30
extractor.sampling_rate:  16000
extractor.n_samples:  480000
extractor.nb_max_frames:  3000
extractor.hop_length:  160
extractor.chunk_length:  30
extractor.sampling_rate:  16000
extractor.n_samples:  480000
extractor.nb_max_frames:  3000
extractor.hop_length:  160
extractor.chunk_length:  30
extractor.sampling_rate:  16000
extractor.n_samples:  480000
extractor.nb_max_frames:  3000
extractor.hop_length:  160
extractor.chunk_length:  30
extractor.sampling_rate:  16000
extractor.n_samples:  480000
extractor.nb_max_frames:  3000
extractor.hop_length:  160


Map:   0%|                             | 6/2290 [00:00<02:00, 18.94 examples/s]

extractor.chunk_length:  30
extractor.sampling_rate:  16000
extractor.n_samples:  480000
extractor.nb_max_frames:  3000
extractor.hop_length:  160
extractor.chunk_length:  30
extractor.sampling_rate:  16000
extractor.n_samples:  480000
extractor.nb_max_frames:  3000
extractor.hop_length:  160
extractor.chunk_length:  30
extractor.sampling_rate:  16000
extractor.n_samples:  480000
extractor.nb_max_frames:  3000
extractor.hop_length:  160
extractor.chunk_length:  30
extractor.sampling_rate:  16000
extractor.n_samples:  480000
extractor.nb_max_frames:  3000
extractor.hop_length:  160


...

extractor.chunk_length:  30
extractor.sampling_rate:  16000
extractor.n_samples:  480000
extractor.nb_max_frames:  3000
extractor.hop_length:  160
extractor.chunk_length:  30
extractor.sampling_rate:  16000
extractor.n_samples:  480000
extractor.nb_max_frames:  3000
extractor.hop_length:  160
extractor.chunk_length:  30
extractor.sampling_rate:  16000
extractor.n_samples:  480000
extractor.nb_max_frames:  3000
extractor.hop_length:  160


Map:  99%|███████████████████████████▊| 127/128 [00:07<00:00, 16.58 examples/s]

extractor.chunk_length:  30
extractor.sampling_rate:  16000
extractor.n_samples:  480000
extractor.nb_max_frames:  3000
extractor.hop_length:  160
extractor.chunk_length:  30
extractor.sampling_rate:  16000
extractor.n_samples:  480000
extractor.nb_max_frames:  3000
extractor.hop_length:  160
extractor.chunk_length:  30
extractor.sampling_rate:  16000
extractor.n_samples:  480000
extractor.nb_max_frames:  3000
extractor.hop_length:  160
extractor.chunk_length:  30
extractor.sampling_rate:  16000
extractor.n_samples:  480000
extractor.nb_max_frames:  3000
extractor.hop_length:  160


Map: 100%|████████████████████████████| 128/128 [00:08<00:00, 15.67 examples/s]

Columns in train set after: ['input_features', 'labels']


In [12]:
def get_shapes(dataset_dict):
    # Just fetch one example from any of the datasets (train, validation, test)
    one_example = next(iter(dataset_dict["train"]))
    input_features_shape = len(one_example["input_features"])  # length for a 1D list
    if isinstance(one_example["input_features"][0], list):  # check if it's 2D
        input_features_shape = (input_features_shape, len(one_example["input_features"][0]))
    
    labels_shape = len(one_example["labels"])  # Assuming labels is a list. Use `.shape` if it's a tensor.
    return input_features_shape, labels_shape

# Assuming dataset_dict is your DatasetDict object with 'prepare_dataset' function already applied
input_features_shape, labels_shape = get_shapes(dataset_dict)

print(f"'input_features' shape: {input_features_shape}")
print(f"'labels' shape: {labels_shape}")


'input_features' shape: (80, 3000)
'labels' shape: 7


In [13]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch    

In [14]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [15]:
import evaluate

metric = evaluate.load("wer")

In [16]:
from sklearn.metrics import confusion_matrix
import numpy as np

true_labels = []
predicted_labels = []

def compute_metrics(pred):
    global true_labels, predicted_labels  # Declare as global to modify

    # For WER
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    # For confusion matrix
    pred_ids_cm = pred.predictions.argmax(-1)
    
    print("Shape of pred_ids_cm:", pred_ids_cm.shape)
    print("Shape of label_ids:", label_ids.shape)
    print("Pad token ID:", processor.tokenizer.pad_token_id)

    # Flatten Arrays
    pred_ids_cm = pred_ids_cm.flatten()
    label_ids = label_ids.flatten()

    # Create and apply mask
    mask = label_ids != processor.tokenizer.pad_token_id
    print("Shape of mask:", mask.shape)

    # pred_ids_cm = pred_ids_cm[mask]
    # label_ids_cm = label_ids[mask]
    
    # print("Shape of masked pred_ids_cm:", pred_ids_cm.shape)
    # print("Shape of masked label_ids:", label_ids_cm.shape)

    # Extend the global lists
    # true_labels.extend(label_ids_cm.tolist())
    # predicted_labels.extend(pred_ids_cm.tolist())

    # # You can compute the confusion matrix here or later
    # cm = confusion_matrix(true_labels, predicted_labels)

    return {"wer": wer}
    # , "confusion matrix": cm}


In [17]:
#Loss & Accuracy curve
#Learning curve
# Define a custom callback to save logs
class MetricsCallback(TrainerCallback):
    def __init__(self):
        self.train_loss_set = []
        self.eval_loss_set = []
        self.steps = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if "loss" in logs.keys():  # Training loss
            self.train_loss_set.append(logs["loss"])
            self.steps.append(state.global_step)
        if "eval_loss" in logs.keys():  # Eval loss
            self.eval_loss_set.append(logs["eval_loss"])

# Initialize callback
metrics_callback = MetricsCallback()


In [18]:
# 4. To store gradient norms
gradient_norms = []

class GradientNormsCallback(TrainerCallback):
    def on_backward_end(self, args, state, control, model=None, **kwargs):
        total_norm = 0
        for p in model.parameters():
            if p.grad is not None:
                param_norm = p.grad.data.norm(2)
                total_norm += param_norm.item() ** 2
        total_norm = total_norm ** (1. / 2)
        gradient_norms.append(total_norm)

# Initialize gradient norms callback
gradient_norms_callback = GradientNormsCallback()

In [19]:
# 5 & 6. To store precision-recall and ROC values
y_true = []
y_pred = []

class EvalPredictionCallback(TrainerCallback):
    def on_prediction_end(self, args, state, control, predictions, label_ids, metrics, **kwargs):
        # Assuming a binary classification problem; 
        # you may need to adjust for your specific case
        preds = torch.softmax(predictions.predictions, dim=-1)[:, 1]
        y_pred.extend(preds.cpu().numpy())
        y_true.extend(label_ids.cpu().numpy())

# Initialize eval callback
eval_pred_callback = EvalPredictionCallback()

In [30]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-1st-training-test",  # change to a repo name of your choice
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=50, #Number of steps used for a linear warmup from 0 to learning_rate. Overrides any effect of warmup_ratio.
    generation_max_length=64, #evaluation:  The max_length to use on each evaluation loop when
    eval_steps=64, #evaluation: Number of update steps between two evaluations if evaluation_strategy="steps".
    max_steps=256, #defaults to -1 — 
                    #If set to a positive number, the total number of training steps to perform. 
                    #Overrides num_train_epochs. 
                    #In case of using a finite iterable dataset the training may stop before reaching the set number of steps when all data is exhausted.
    save_steps=64, #during training, intermediate checkpoints will be saved and uploaded asynchronously to the Hub every save_steps training steps
    logging_steps=10,
    # gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    predict_with_generate=True,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

In [31]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[metrics_callback, gradient_norms_callback, eval_pred_callback]
)

In [32]:
processor.save_pretrained(training_args.output_dir)

In [33]:
trainer.train()

Step,Training Loss,Validation Loss



KeyboardInterrupt



In [ ]:
# conf_matrix = confusion_matrix(true_labels, predicted_labels)
# print("Confusion Matrix:")
# print(conf_matrix)

# import seaborn as sns

# plt.figure(figsize=(10, 7))
# sns.heatmap(conf_matrix, annot=True, fmt="d")
# plt.xlabel('Predicted')
# plt.ylabel('True')
# plt.show()

In [ ]:
# #Loss Curve: Plotting the training and validation loss helps you understand how well the model is learning. 
# # You want the loss to decrease over time without the validation loss increasing (which could indicate overfitting)

# #Accuracy Curve: If applicable, plotting training and validation accuracy can provide insights into the model's performance. 
# # However, for tasks like machine translation or text-to-speech, specialized metrics might be more useful.
# #Loss & Accuracy curve
# # Now plot the saved metrics
# plt.figure()
# plt.plot(metrics_callback.steps, metrics_callback.train_loss_set, label="Training Loss")
# plt.plot(metrics_callback.steps, metrics_callback.eval_loss_set, label="Validation Loss")
# plt.title("Loss Curve")
# plt.xlabel("Steps")
# plt.ylabel("Loss")
# plt.legend()
# plt.show()


In [ ]:
# #Learning Rate: If you're using a learning rate schedule, plotting the learning rate over time can be useful.
# #Learning curve
# # This will serve as a makeshift learning curve
# plt.figure()
# plt.plot(metrics_callback.steps, metrics_callback.train_loss_set, label="Training Loss")
# plt.title("Learning Curve")
# plt.xlabel("Steps")
# plt.ylabel("Training Loss")
# plt.legend()
# plt.show()


In [ ]:
# #Gradient Norms: Monitoring the norms of gradients can help in diagnosing issues related to 
# # vanishing or exploding gradients.
# # 4. Plot gradient norms
# plt.figure()
# plt.plot(range(len(gradient_norms)), gradient_norms)
# plt.title('Gradient Norms')
# plt.xlabel('Training Steps')
# plt.ylabel('Gradient Norm')
# plt.show()


In [ ]:
# #Precision-Recall Curve: Especially useful for imbalanced datasets, 
# # this curve helps you understand the trade-off between precision and recall for your model.
# # 5. Plot Precision-Recall curve
# precision, recall, _ = precision_recall_curve(y_true, y_pred)
# plt.figure()
# plt.plot(recall, precision)
# plt.title('Precision-Recall Curve')
# plt.xlabel('Recall')
# plt.ylabel('Precision')
# plt.show()


In [ ]:
# #ROC Curve and AUC: Receiver operating characteristic curve is a graphical plot that 
# # illustrates the diagnostic ability of a binary classifier.
# # 6. Plot ROC curve
# fpr, tpr, _ = roc_curve(y_true, y_pred)
# roc_auc = roc_auc_score(y_true, y_pred)
# plt.figure()
# plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
# plt.title('ROC Curve')
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.legend()
# plt.show()

# #######


In [ ]:
# from transformers import pipeline
# import gradio as gr

# pipe = pipeline(model="./whisper-1st-training-test")  # change to "your-username/the-name-you-picked"

# def transcribe(audio):
#     text = pipe(audio)["text"]
#     return text

# iface = gr.Interface(
#     fn=transcribe, 
#     inputs=gr.Audio(source="microphone", type="filepath"), 
#     outputs="text",
#     title="Whisper tiny healthcare",
#     description="Realtime demo for healthcare speech recognition using a fine-tuned Whisper tiny model.",
# )

# iface.launch()